# Match BGBM Collectors to Wikidata Items Using *Nearest Neighbour*

Basically we attempt a match of `canonical_string` of WikiData to `canonical_string` of the collectors (in this case the names were parsed beforehand into single names using <https://libraries.io/rubygems/dwc_agent>).

See also [match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_k-nearest.ipynb](./match_names_BGBM-dwcagent-parsed-eventDate_vs_WikiData_k-nearest.ipynb) that includes the event date of sampling to get a time reference for deciding name matches to WikiData.

TODO:

- review code: match when full collector name was given instead of an abbreviated collector name
- review code: evaluate if multiple names (WikiData or collector data) are found
- match also with time periode of work (WikiData) ⇌ created time of the herbarium sheet (if no other life time data are available)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
import pprint, time, os
wikidata = pd.read_csv("data/wikidata_persons_botanists_20230703_1352.csv", index_col=0, low_memory=False)

wikidata.head()

,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768.0,1826.0,NaN,NaN,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818.0,1904.0,NaN,NaN,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718.0,1790.0,NaN,NaN,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805.0,1860.0,NaN,NaN,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808.0,1880.0,NaN,NaN,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# Create data frame with unique canonical strings 
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()

wd_matchtest

# colls = list(wikidata.columns)

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


In [3]:
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

wd_matchtest_fullnames


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


### Load Collectors Data Set

Data sources:

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

TODO:
- check parsed fields `particle` and other fields, e.g. «`Abbas al Ani, H.`» to get a proper abbreviated atomized name

    ```bash
    cd data/VHde_doi-10.15468-dl.tued2e/
    head occurrence_recordedBy_occurrenceIDs_20230524_parsed.tsv \
      | column --table --separator $'\t' \
      | sed 's@^@  # @;'
      # family     given  suffix  particle  dropping_particle  nick  appellation  title  occurrenceID_count  occurrenceID_first
      #            No                                                                    1                   http://id.snsb.info/snsb/collection/108286/167064/109352
      # Azofeifa   A.                                                                    2                   https://herbarium.bgbm.org/object/B200211416
      # A. Cano    E.                                                                    1                   https://herbarium.bgbm.org/object/B100699397
      # Henry      A.                                                                    1                   https://herbarium.bgbm.org/object/B200098813
      # Selmons    Ad                                                                    1                   https://herbarium.bgbm.org/object/B100379213
      # Aaronsohn  A.                                                                    3                   https://je.jacq.org/JE00010154
      ```


In [4]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_occurrenceIDs_20230524_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.
collectors.sort_values(by=['family', 'given','occurrenceID_first'], inplace=True)
collectors

,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first
1,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100699397
39713,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B101149305
4,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://je.jacq.org/JE00010154
26947,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://herbarium.bgbm.org/object/B100217620
26951,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,2,https://herbarium.bgbm.org/object/B100326682
...,...,...,...,...,...,...,...,...,...,...
66493,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,5,https://herbarium.bgbm.org/object/B100263330
32807,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100263331
66494,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,3,https://herbarium.bgbm.org/object/B100009590
66495,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100530714


#### Check Composition of Parsed Collector Data

In [5]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (1365 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first
2207,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B200125981
27038,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100184021
21012,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,1,http://id.snsb.info/snsb/collection/22086/3086...
18442,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100296455
412,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100194646
...,...,...,...,...,...,...,...,...,...,...
65711,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,3,https://je.jacq.org/JE04006629
65712,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,1,https://je.jacq.org/JE04007139
65797,Ziz,Johann Baptist,NaN,NaN,NaN,NaN,NaN,NaN,1,https://je.jacq.org/JE00017744
65859,Zollinger,Heinrich,NaN,NaN,NaN,NaN,NaN,NaN,2,https://herbarium.bgbm.org/object/B101097046


In [6]:
# check the parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head())


----------------------------------------
show names with **particle** found 694 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first
21012,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,1,http://id.snsb.info/snsb/collection/22086/3086...
360,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN,96,https://herbarium.bgbm.org/object/B100576238
4090,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN,4,https://herbarium.bgbm.org/object/B100031063
60797,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN,26,https://herbarium.bgbm.org/object/B100031454
16744,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN,2,https://herbarium.bgbm.org/object/B100031644



----------------------------------------
show names with **suffix** found 15 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first
17265,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN,21,https://dr.jacq.org/DR014960
58842,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN,1,https://je.jacq.org/JE04008848
16996,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN,1,http://id.snsb.info/snsb/collection/504525/625...
800,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN,1,https://herbarium.bgbm.org/object/B100525791
26158,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN,2,https://herbarium.bgbm.org/object/B100525792



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title,occurrenceID_count,occurrenceID_first
17097,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN,2,https://herbarium.bgbm.org/object/B100127256


Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [7]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)
collectors.tail()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_count,occurrenceID_first
66493,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",5,https://herbarium.bgbm.org/object/B100263330
32807,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",1,https://herbarium.bgbm.org/object/B100263331
66494,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.",3,https://herbarium.bgbm.org/object/B100009590
66495,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",1,https://herbarium.bgbm.org/object/B100530714
66496,Жирова,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Жирова, O.",1,https://herbarium.bgbm.org/object/B100630811


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_first', lambda x: list(x)[0]) # custom function, to get the first entry
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

collectors_unique

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.",1,https://herbarium.bgbm.org/object/B100699397
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,1,https://herbarium.bgbm.org/object/B101149305
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,https://je.jacq.org/JE00010154
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.",5,https://herbarium.bgbm.org/object/B100217620
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.",1,https://herbarium.bgbm.org/object/B101153811
...,...,...,...,...,...,...,...,...,...,...,...
20817,Żelazny,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelazny, J.",4,https://herbarium.bgbm.org/object/B100344466
20818,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.",6,https://herbarium.bgbm.org/object/B100263330
20819,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.",3,https://herbarium.bgbm.org/object/B100009590
20820,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",1,https://herbarium.bgbm.org/object/B100530714


In [9]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [10]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))

(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(1835-1906), G.A.F.E. = (1835-1906), Gustav Adolf Ferdinand Eichler'
'(1873-1926), S.S. = (1873-1926), Søren Sørensen'
'(1888–1973), G.A. = (1888–1973), Georges André'
'(1904-1990), J.J. = (1904-1990), Johannes Johannessen'


In [11]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD', r'', string)
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [12]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))

Show ngram examples:
- simple name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- data from collectors: [' Aa', 'Aai', 'aik', 'iki', 'ki ']
- data from match-test: [' Wa', 'Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' Oh', 'Oh ']
- data from match-test (full name): [' 18', '188', '888', '881', '819', '197', '973', '73 ', '3 G', ' Ge', 'Geo', 'eor', 'org', 'rge', 'ges', 'es ', 's A', ' An', 'And', 'ndr', 'dr ']


Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (BGBM) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [13]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step


def getNearestNeighbour(query, this_vectorizer, this_nbrs_data):
    """Calculate the k-nearest distance for query data using package scikit-learn


    @param query: DataFrame the query data to vectorize and transform
    @param this_vectorizer: the vectorizer of TfidfVectorizer
    @param this_nbrs_data: the data of NearestNeighbors calculations
    @return: (distances, indices) distances and indices
    @rtype (int, int)
    """
    queryTFIDF_ = this_vectorizer.transform(query)
    distances, indices = this_nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: Number of neighbors required for each sample by default for :meth:`kneighbors` queries (originally 5).

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """

    import time
    start = time.time()
    query_data = set(query_data)
    # convert list to set for better performance

    print('Vectorizing data. This may take a while...')
    # vectorize wikidata names
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_vector_data = vectorizer.fit_transform(match_data
        # wd_matchtest['canonical_string']
    )
    nbrs_data = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1).fit(tfidf_vector_data)
    duration = time.time() - start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    print('Getting nearest neighbours of %s data with %s neighbor sample(s)...' % (len(query_data), n_neighbors))
    distances, indices = getNearestNeighbour(query_data, vectorizer, nbrs_data)
    duration = time.time() - start
    print('Completed after %s s' % duration)

    query_data = list(query_data)  # convert back to list

    print('Finding matches build new data frame ...')
    matches = []
    for i, j in enumerate(indices):
        temp = [query_data[i], match_data.values[j][0], round(distances[i][0], 2)]
        matches.append(temp)

    duration = time.time() - start
    print('Building matches done after %s s' % duration)
    matches = pd.DataFrame(
        matches,
        columns=['namematch_source_data', 'namematch_resource_data', 'namematch_distance']
    )

    print('Done')
    return matches


In [14]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
print("Calculate abbrevated names only …")
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

matches

Calculate abbrevated names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.463573694229126 s
Getting nearest neighbours of 20291 data with 5 neighbor sample(s)...
Completed after 138.69559502601624 s
Finding matches build new data frame ...
Building matches done after 138.8878116607666 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,10145,"Breitwieser, I.","Breitwieser, I.",0.00
1,10617,"Caflisch, J.F.","Caflisch, J.F.",0.00
2,10628,"Davidson, J.","Davidson, J.",0.00
3,10640,"John, H.S.","John, H.S.",0.00
4,10642,"Sedgwick, L.J.","Sedgwick, L.J.",0.00
...,...,...,...,...
20286,19908,"Lescrauwaet, E.","Lescot, M.",1.22
20287,2209,Kyusoz,"Nyushko, T.I.",1.22
20288,19226,"Avjetijezs, V.","Mekhtijeva, N.A.",1.23
20289,13012,Zulptuwaski,"Załęski, E.",1.24


In [15]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
print("Calculate full names only …")
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

matches_fullnames


Calculate full names only …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 4.047614336013794 s
Getting nearest neighbours of 531 data with 5 neighbor sample(s)...
Completed after 13.50185251235962 s
Finding matches build new data frame ...
Building matches done after 13.51045846939087 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,369,"Kunze, Johannes","Kunze, Johannes",0.00
1,97,"Koch, Walo","Koch, Walo",0.00
2,98,"Buchanan, John","Buchanan, John",0.00
3,448,"Evers, Georg","Evers, Georg",0.00
4,100,"Maguire, Bassett","Maguire, Bassett",0.00
...,...,...,...,...
526,114,"Entry, Provisional","Şentürk, Ozan",1.20
527,290,"Kirinco, Synho","Synge, Hugh",1.20
528,222,"Rainbird, Coldecott","Colden, Jane",1.21
529,69,"Xinjiang, Inst Bio-Ped-Psamm","Insam, J",1.21


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

In [16]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.",1,https://herbarium.bgbm.org/object/B100699397,19351,"A. Cano, E.","Cano, Á.",0.64
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,1,https://herbarium.bgbm.org/object/B101149305,6867,Aaiki,"Naiki, A.",0.84
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,https://je.jacq.org/JE00010154,17122,"Aaronsohn, A.","Aaronsohn, A.",0.00
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.",5,https://herbarium.bgbm.org/object/B100217620,9051,"Abaouz, A.","Arbaoui, S.",1.12
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.",1,https://herbarium.bgbm.org/object/B101153811,15038,"Abarca, R.","Abarca, L.",0.57


In [17]:
# append full name matches
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Abdallah, Raffael",1,https://herbarium.bgbm.org/object/B200125981,404,"Abdallah, Raffael","Issel, Raffaele",1.04
1,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,"Abdul, Kadir Bin",1,https://herbarium.bgbm.org/object/B100184021,187,"Abdul, Kadir Bin","Kadir, Zaid bin A",1.00
2,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,"Abreu, Guilherme",1,http://id.snsb.info/snsb/collection/22086/3086...,519,"Abreu, Guilherme","Rau, Guilherme",0.81
3,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"Adá, García",1,https://herbarium.bgbm.org/object/B100296455,20,"Adá, García","Adá, Ramón García",0.78
4,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers",1,https://herbarium.bgbm.org/object/B100194646,151,"Ahagen, Schiers","Séchier, P",1.07


In [18]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,old_index,namematch_source_data,namematch_resource_data,namematch_distance
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,https://je.jacq.org/JE00010154,17122,"Aaronsohn, A.","Aaronsohn, A.",0.0
7,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",2,https://herbarium.bgbm.org/object/B100241637,5265,"Abbe, E.C.","Abbe, E.C.",0.0
11,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.",80,https://herbarium.bgbm.org/object/B100181131,19732,"Abbott, J.R.","Abbott, J.R.",0.0
13,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.",4,http://id.snsb.info/snsb/collection/504820/626...,894,"Abbott, W.L.","Abbott, W.L.",0.0
19,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.",14,https://herbarium.bgbm.org/object/B100046632,4250,"Abedin, S.","Abedin, S.",0.0


In [19]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

Save the plain name matching results only ...

In [20]:
if not os.path.exists('data'):
    print("Make data directory for saving …")
    os.makedirs('data')

# Set some global varialbes
# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20230823

this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
    this_timestamp_for_data
)

collectors_all_matches.to_csv(this_output_file)

print("Wrote plain name matches of collector names into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote plain name matches of collector names into data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_plain-names_20230823.csv (2476 kB)


### Merge and Aggregate Matched Data and WikiData’s

Review (TODO)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [21]:
# merge now the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [22]:
print("Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
print("Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …")
print("Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …")
print("      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)")

criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
collectors_matches_g1_merged_wikidata[criterion].get([
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'itemLabel', 'wikidata_link'
])

Show example data of «Kotschy…» with namematch distances from 0.0 to almost 1.0 (in Cosine Similiarity we had 0.5 … 1.0)
Interpretation: most of the matches seem correct (=Carl Georg Theodor) also with higher distances but we cannot be sure in mid ranges …
Interestingly with an …yi: Kotschyi, C.G.T. (0.92) was matched to Carl Georg Theodor (=correct, and higher distance) …
      … but Kotschy, C.G.T. (0.76) was calculated to the other person Carl Friedrich (=incorrect, lower distance … :-/)


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,wikidata_link
10701,2,https://dr.jacq.org/DR049432,Kotschy,"Kotschy, T.",0.43,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10702,37,http://id.snsb.info/snsb/collection/16719/2549...,"Kotschy, K.G.T.","Kotschy, T.",0.72,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10703,310,http://id.snsb.info/snsb/collection/117808/176...,"Kotschy, T.","Kotschy, T.",0.00,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10704,5,https://herbarium.bgbm.org/object/B100526350,"Kotschy, Th","Kotschy, T.",0.50,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10705,1,https://herbarium.bgbm.org/object/B100160086,"Kotschyi, C.G.T.","Kotschy, T.",0.92,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
10707,1,http://id.snsb.info/snsb/collection/22980/3175...,"Kotschy, C.G.","Kotschy, C.F.",0.63,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
10708,2494,http://id.snsb.info/snsb/collection/108230/167...,"Kotschy, C.G.T.","Kotschy, C.F.",0.76,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
22091,2,https://je.jacq.org/JE00022436,"Kotschy, Carl Georg Theodor","Kotschy, Theodor",0.68,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
22092,1,https://herbarium.bgbm.org/object/B101113772,"Kotschy, Karl Georg Th","Kotschy, Theodor",0.98,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299


Aggregate data to get atomized listings of multiple resource name matches joining by “|” aso.

In [23]:
print('Group data by canonical names (abbreviated and full name): '
      '1st multiple related WD items (e.g. Q1232456), 2nd multiple WD item labels')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
    print('Run %s: Group by wiki data’s %s, and aggregate/join item(s) '
          'by “…|…”, add new column items_joined ...' % (i + 1, wd_matching_column))
    wdata_joined_items = wikidata.groupby([wd_matching_column])['item'].apply('|'.join).reset_index()
    wdata_joined_items.rename(columns={'item': 'items_joined'}, inplace=True)

    # print("Done. Show examples of items having multiple matching data «|» … ")
    # criterion = wdata_joined_items['items'].map(lambda x: '|' in x)
    # wdata_joined_items[criterion].head()

    print('Run %s:   Merge all based on namematch_resource_data, add item(s) data ...' % (i + 1))
    collectors_matches_g2 = pd.merge(
        collectors_matches_g1_merged_wikidata, wdata_joined_items,
        left_on='namematch_resource_data', right_on=wd_matching_column
        , suffixes=('__wikidata_merge', '__grp_by_items')
        # append to left-data, right-data only when identical column names occur
    )

    # link wikidata items with canonical name string (pipe separated if more than one)
    print('Run %s:   Group by wiki data’s %s, and aggregate/join itemLabel(s) '
          'by “…|…”, add new column item_labels_joined ...' % (i + 1, wd_matching_column))
    wdata_joined_itemlabels = wikidata.groupby([wd_matching_column])['itemLabel'].apply('|'.join).reset_index()
    wdata_joined_itemlabels.rename(columns={'itemLabel': 'item_labels_joined'}, inplace=True)

    print('Run %s:   Merge all based on namematch_resource_data, add itemLabel(s) data ...' % (i + 1))
    collectors_matches_g3 = pd.merge(
        collectors_matches_g2, wdata_joined_itemlabels,
        left_on='namematch_resource_data', right_on=wd_matching_column
        , suffixes=('__grp_by_items', '__grp_by_item_labels')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_group” ...' % (i + 1))
    collectors_matches_group = collectors_matches_g3 \
        if i == 0 \
        else pd.concat([collectors_matches_group, collectors_matches_g3], ignore_index = True)
    
print('Done')

Group data by canonical names (abbreviated and full name): 1st multiple related WD items (e.g. Q1232456), 2nd multiple WD item labels
Run 1: Group by wiki data’s canonical_string, and aggregate/join item(s) by “…|…”, add new column items_joined ...
Run 1:   Merge all based on namematch_resource_data, add item(s) data ...
Run 1:   Group by wiki data’s canonical_string, and aggregate/join itemLabel(s) by “…|…”, add new column item_labels_joined ...
Run 1:   Merge all based on namematch_resource_data, add itemLabel(s) data ...
Run 1:   Build data frame “collectors_matches_group” ...
Run 2: Group by wiki data’s canonical_string_fullname, and aggregate/join item(s) by “…|…”, add new column items_joined ...
Run 2:   Merge all based on namematch_resource_data, add item(s) data ...
Run 2:   Group by wiki data’s canonical_string_fullname, and aggregate/join itemLabel(s) by “…|…”, add new column item_labels_joined ...
Run 2:   Merge all based on namematch_resource_data, add itemLabel(s) data ...

In [24]:
print("Show examples of item_labels_joined having multiple matching data «|» … ")
criterion = collectors_matches_group['item_labels_joined'].map(lambda x: '|' in x)

collectors_matches_group[criterion].get([ # empty 
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    # 'canonical_string_fullname', 
    'item_labels_joined', 'items_joined'
], default="...get: Are data empty or it has probably a wrong named column?")


Show examples of item_labels_joined having multiple matching data «|» … 


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,item_labels_joined,items_joined
8,1,https://herbarium.bgbm.org/object/B600197578,"Abbasju, M.","Abbas, A.",0.95,Alia Abbas|Abdulla Abbas,http://www.wikidata.org/entity/Q60141229|http:...
9,1,https://herbarium.bgbm.org/object/B600197578,"Abbasju, M.","Abbas, A.",0.95,Alia Abbas|Abdulla Abbas,http://www.wikidata.org/entity/Q60141229|http:...
22,4,https://herbarium.bgbm.org/object/B100005623,"Abe, L.M.","Abe, J.",0.85,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...
23,4,https://herbarium.bgbm.org/object/B100005623,"Abe, L.M.","Abe, J.",0.85,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...
24,1,https://herbarium.bgbm.org/object/B100847062,"Abeinaen, B.","Abe, J.",1.14,Jun Abe|Junichiro Abe,http://www.wikidata.org/entity/Q112507805|http...
...,...,...,...,...,...,...,...
22342,6,https://dr.jacq.org/DR030609,"Müller, Hermann","Müller, Hermann",0.00,Hermann Müller|Hermann Müller,http://www.wikidata.org/entity/Q68738|http://w...
22359,6,https://herbarium.bgbm.org/object/B100241696,"Palmer, Edw","Palmer, Edward",0.74,Edward Palmer|Edward Palmer,http://www.wikidata.org/entity/Q3048655|http:/...
22360,6,https://herbarium.bgbm.org/object/B100241696,"Palmer, Edw","Palmer, Edward",0.74,Edward Palmer|Edward Palmer,http://www.wikidata.org/entity/Q3048655|http:/...
22513,1,https://herbarium.bgbm.org/object/BGT0002482,"Xu, Honggui","Xu, Han",1.00,Han Xu|Han Xu,http://www.wikidata.org/entity/Q21613168|http:...


In [25]:
# check what columns we have and what we would keep for further analysis and what to drop
pprint.pprint(collectors_matches_group.columns)
# from merge: _x would means normally from left column, _y means from right column
# in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@#  @'

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'old_index', 'namematch_source_data', 'namematch_resource_data',
       'namematch_distance', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string__wikidata_merge', 'canonical_string_fullname',
       'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob',
       'yod', 'wyb', 'wye', 'wikidata_link', 'orcid_link', 'harv_link',
       'ipni_link', 'bionomia_link', 'canonical_string__grp_by_items',
       'items_joined', 'canonical_string', 'item_labels_joined',
       'canonical_string_fullname__wikidata_merge',
       'canonical_string_fullname__grp_by_items'],
      dtype='object')


Prepare data to save later on …

In [26]:
# Remove superfluous columns TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# TODO check duplicates
collectors_matches_group_simplified = collectors_matches_group.get(
    ['family', 'given', 'canonical_string_collector_parsed', 
      'namematch_source_data', # redundant: 'namematch_source_data' == 'canonical_string_collector_parsed'
     'namematch_resource_data', 'namematch_distance', 
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye',
    'items_joined', 'canonical_string', 'canonical_string_fullname', 'surname', 'initials', 'item_labels_joined'
    ], default="...get: Are data empty or it has probably a wrong named column?"
)
# collectors_matches_group = collectors_matches_g3
collectors_matches_group_simplified.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_group_simplified.drop_duplicates(inplace=True)
collectors_matches_group_simplified.head()

/tmp/ipykernel_36438/2558052948.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.sort_values(
/tmp/ipykernel_36438/2558052948.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_matches_group_simplified.drop_duplicates(inplace=True)


,family,given,canonical_string_collector_parsed,namematch_source_data,namematch_resource_data,namematch_distance,orcid,viaf,isni,harv,...,yob,yod,wyb,wye,items_joined,canonical_string,canonical_string_fullname,surname,initials,item_labels_joined
4,Aaronsohn,A.,"Aaronsohn, A.","Aaronsohn, A.","Aaronsohn, A.",0.0,NaN,2795076,0000 0001 0948 8581,30592,...,1876.0,1919.0,NaN,NaN,http://www.wikidata.org/entity/Q2086130,"Aaronsohn, A.","Aaronsohn, Aaron",Aaronsohn,A.,Aaron Aaronsohn
10,Abbe,E.C.,"Abbe, E.C.","Abbe, E.C.","Abbe, E.C.",0.0,NaN,101473381,0000 0000 7237 8505,30066,...,1905.0,2000.0,NaN,NaN,http://www.wikidata.org/entity/Q10274118,"Abbe, E.C.","Abbe, Ernst Cleveland",Abbe,E.C.,Ernst Cleveland Abbe
14,Abbott,J.R.,"Abbott, J.R.","Abbott, J.R.","Abbott, J.R.",0.0,NaN,NaN,NaN,NaN,...,1968.0,NaN,NaN,NaN,http://www.wikidata.org/entity/Q18982386,"Abbott, J.R.","Abbott, J Richard",Abbott,J.R.,J. Richard Abbott
16,Abbott,W.L.,"Abbott, W.L.","Abbott, W.L.","Abbott, W.L.",0.0,NaN,1545420,0000 0000 3712 5377,27518,...,1860.0,1936.0,NaN,NaN,http://www.wikidata.org/entity/Q635604,"Abbott, W.L.","Abbott, William Louis",Abbott,W.L.,William Louis Abbott
26,Abedin,S.,"Abedin, S.","Abedin, S.","Abedin, S.",0.0,NaN,5859151837993620520007,NaN,69097,...,1952.0,NaN,NaN,NaN,http://www.wikidata.org/entity/Q16142861,"Abedin, S.","Abedin, Sultanul",Abedin,S.,Sultanul Abedin


In [27]:
# old file bgbm_collectors_matches_wikidata_items_group_concat_%s.csv
this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_%s.csv' % (
    this_timestamp_for_data
)

collectors_matches_group.to_csv(this_output_file)

print("Wrote groups of collectors matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote groups of collectors matches into data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_20230823.csv (10950 kB)


### Merge Data to Individual WikiData Items

For this, merge by namematch_resource_data and focus to get individual WikiData items.

In [28]:
print('Merge simply namematch_resource_data to Wiki data for abbreviated and full names... ')
for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):

    # join wikidata items to avh collectors matches
    #   avh_matches = pd.merge(avh, matches, left_on='label', right_on='name')
    #   avh_matches_t1 = pd.merge(avh_matches, wikidata, left_on='matched_name', right_on='canonical_string')
    # link counts of wikidata items with same canonical name string
    #   avh_matches_t2 = pd.merge(avh_matches_t1, wd_test, left_on="matched_name", right_on="canonical_string")
    #   avh_matches_t2.rename(columns = {list(avh_matches_t2.columns)[-1]: 'dup_count'}, inplace=True)
    
    print('Run %s:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...' % (i + 1))
    collectors_matches_wd1 = pd.merge(
        collectors_all_matches, wikidata,
        left_on='namematch_resource_data', right_on=wd_matching_column,
        suffixes=('__coll_all_matches', '__wd')
        # append to left-data, right-data only when identical column names occur
    )

    print('Run %s:   Build data frame “collectors_matches_with_wdata” ...' % (i + 1))
    collectors_matches_with_wdata = collectors_matches_wd1 \
        if i == 0 \
        else pd.concat([collectors_matches_with_wdata, collectors_matches_wd1], ignore_index=True)

print('Done')


Merge simply namematch_resource_data to Wiki data for abbreviated and full names... 
Run 1:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...
Run 1:   Build data frame “collectors_matches_with_wdata” ...
Run 2:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...
Run 2:   Build data frame “collectors_matches_with_wdata” ...
Done


In [29]:
pprint.pprint(collectors_matches_with_wdata.columns)
# echo "${text}" | fold --spaces | sed 's@^@#  @'

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
       'old_index', 'namematch_source_data', 'namematch_resource_data',
       'namematch_distance', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [30]:
collectors_matches_with_wdata.drop_duplicates(inplace=True)
collectors_matches_with_wdata

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,occurrenceID_collectors_count,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.",3,...,Q2086130,1876.0,1919.0,NaN,NaN,http://www.wikidata.org/wiki/Q2086130,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23-1,https://bionomia.net/Q2086130
1,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.",2,...,Q10274118,1905.0,2000.0,NaN,NaN,http://www.wikidata.org/wiki/Q10274118,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/26-1,https://bionomia.net/Q10274118
2,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.",80,...,NaN,1968.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q18982386,NaN,NaN,https://www.ipni.org/a/20015671-1,NaN
3,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.",4,...,Q635604,1860.0,1936.0,NaN,NaN,http://www.wikidata.org/wiki/Q635604,NaN,https://kiki.huh.harvard.edu/databases/botanis...,NaN,https://bionomia.net/Q635604
4,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.",14,...,NaN,1952.0,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q16142861,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/35239-1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22520,Entry,Provisional,NaN,NaN,NaN,NaN,NaN,NaN,"Entry, Provisional",152,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q82758799,NaN,NaN,https://www.ipni.org/a/20040762-1,NaN
22521,Kirinco,Synho,NaN,NaN,NaN,NaN,NaN,NaN,"Kirinco, Synho",1,...,NaN,1951.0,2018.0,NaN,NaN,http://www.wikidata.org/wiki/Q56742818,NaN,https://kiki.huh.harvard.edu/databases/botanis...,NaN,NaN
22522,Rainbird,Coldecott,NaN,NaN,NaN,NaN,NaN,NaN,"Rainbird, Coldecott",1,...,Q2504068,1724.0,1766.0,NaN,NaN,http://www.wikidata.org/wiki/Q2504068,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/1730-1,https://bionomia.net/Q2504068
22523,Xinjiang,Inst Bio-Ped-Psamm,NaN,NaN,NaN,NaN,NaN,NaN,"Xinjiang, Inst Bio-Ped-Psamm",2,...,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/wiki/Q36592097,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/29635-1,NaN


Save all columns for further analysis

In [31]:
# old bgbm_collectors_matches_wikidata-botanists_all-columns_%s.csv

this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv' % (
    this_timestamp_for_data
)

collectors_matches_with_wdata.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)
collectors_matches_with_wdata.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items of collector matches into data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_20230823.csv (8569 kB)


In [32]:
# TODO meaningful?
# remove redundant (duplicate (?or empty?)) columns that in any kind are duplicate data (i.e. that we usually do not need)
# do it by transposing it (https://www.statology.org/pandas-drop-duplicate-columns/)
compact_df_tmp=collectors_matches_with_wdata.transpose().drop_duplicates().transpose()
compact_df_tmp.sort_values(
    by=['namematch_distance', 'canonical_string_collector_parsed']
    , inplace=True
)

# old bgbm_collectors_matches_wikidata-botanists_all-columns-made-unique_%s.csv
# results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv
this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_%s.csv' % (
    # "20230823"
    time.strftime('%Y%m%d') # '%Y%m%d_%H%M'
)

compact_df_tmp.to_csv(
    this_output_file, index=False # drop index column
)

print("Wrote isolated WikiData items (unique columns) of collector matches into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote isolated WikiData items (unique columns) of collector matches into data/results_bgbm_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_20230920.csv (8204 kB)


In [33]:
# TODO further evaluation or filtering, counting, clean up aso.

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
TODO … | Year of first collection
TODO end_date | Year of last collection
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
name_match_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest